In [ ]:
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
#!pip install  chart_studio
import seaborn as sns

from functools import reduce
from glob import glob

#ipywidgets for some interactive plots
from ipywidgets.widgets import * 
import ipywidgets as widgets

#plotly 3D interactive graphs 
import plotly
from plotly.graph_objs import *
#import chart_studio

import random
from random import sample
import sklearn.model_selection as skl
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image.dataframe_iterator import DataFrameIterator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import InputLayer, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.metrics import AUC, MeanAbsoluteError, BinaryCrossentropy, BinaryAccuracy
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.losses import BinaryCrossentropy, MeanAbsoluteError ,Hinge
import tensorflow_hub as tfhub
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report

import pydicom
import cv2

# **Forming the Data Frames**
**In the following cells we have split the train data set into train, validation and test.
We have done so because our dataset was part of a competition and we had no means of checking our model's accuracy with the test dataset as it had no labels.**

In [ ]:
root_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
df = pd.read_csv(root_dir+'train_labels.csv')
df.head()

In [ ]:
# Add the full paths for each id for different types of sequences to the csv 
def full_ids(data):
    zeros = 5 - len(str(data))
    if zeros > 0:
        prefix = ''.join(['0' for i in range(zeros)])
    
    return prefix+str(data)
        

df['BraTS21ID_full'] = df['BraTS21ID'].apply(full_ids)

# Add all the paths to the df for easy access
df['flair'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/FLAIR/')
df['t1w'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1w/')
df['t1wce'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1wCE/')
df['t2w'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T2w/')

df.head()

In [ ]:
df, df_test = train_test_split(df,test_size=0.1)

In [ ]:
df.head()

In [ ]:
df_test.head()

In [ ]:
df.describe()

In [ ]:
df_test.describe()

In [ ]:
df.groupby('MGMT_value').count().head()

In [ ]:
df_test.groupby('MGMT_value').count().head()

# Loading the Images

**In the following cells:**
* We are actually splitting the dataset into train, validation and test and we are organising the respective labels
* We are augmenting the data in real time to bring about some variation in the dataset
* We are also using our user defined class to itterate through the dataframe

**Our dataset had a few discrepencies with a few images such as:**
* 00109 (FLAIR images are blank)
* 00123 (T1w images are blank)
* 00709 (FLAIR images are blank)

In [ ]:
def get_train_val_dataframe(mri_type):
    
    all_img_files = []
    all_img_labels = []
    all_img_patient_ids = []
    for row in df.iterrows():
        if row[1]['BraTS21ID_full'] == '00109' and mri_type == 'flair':
            continue
        if row[1]['BraTS21ID_full'] == '00123' and mri_type == 't1w':
            continue
        if row[1]['BraTS21ID_full'] == '00709' and mri_type == 'flair':
            continue
        img_dir = row[1][mri_type]
        img_files = os.listdir(img_dir)
        img_nums = sorted([int(ele.replace('Image-', '').replace('.dcm', '')) for ele in img_files])
        mid_point = int(len(img_nums)/2)
        start_point = mid_point - max(int(mid_point*0.1), 1)
        end_point = mid_point + max(int(mid_point*0.1), 1)
        img_names = [f'Image-{img_nums[i]}.dcm' for i in range(start_point, end_point+1)]
        img_paths = [img_dir+ele for ele in img_names]
        img_labels = [row[1]['MGMT_value']]*len(img_paths)
        img_patient_ids = [row[1]['BraTS21ID']]*len(img_paths)
        all_img_files.extend(img_paths)
        all_img_labels.extend(img_labels)
        all_img_patient_ids.extend(img_patient_ids)

    train_val_df = pd.DataFrame({'patient_ids': all_img_patient_ids,
                  'labels': all_img_labels,
                  'file_paths': all_img_files})

    train_val_df['labels'] = train_val_df['labels'].map({1: '1', 0: '0'})
    
    #stratifiied 85% split on patient_ids and labels  
    class_prop= 0.85
    
    classes_splits  = {}
    for i in range(2):
        train_val_label_class = train_val_df[train_val_df['labels']==f'{i}']
        train_val_list_ids =  list(train_val_label_class['patient_ids'].unique())
        train_threshold = math.ceil(class_prop*len(train_val_list_ids))
        train_ids = train_val_list_ids[:train_threshold]
        val_ids = train_val_list_ids[train_threshold:]
        classes_splits[f'train_{i}'] = train_val_label_class[train_val_label_class['patient_ids'].isin(train_ids)]
        classes_splits[f'val_{i}'] = val_df = train_val_label_class[train_val_label_class['patient_ids'].isin(val_ids)]
        
    train_df = pd.concat([classes_splits['train_0'], classes_splits['train_1']], axis=0)
    val_df = pd.concat([classes_splits['val_0'], classes_splits['val_1']], axis=0)
    
    train_df.head()
    val_df.head()
    return train_df, val_df
    
    
def get_test_dataframe(mri_type):
    
    all_test_img_files = []
    all_test_img_labels = []
    all_test_img_patient_ids = []
    for row in df_test.iterrows():
        img_dir = row[1][mri_type]
        img_files = os.listdir(img_dir)
        img_nums = sorted([int(ele.replace('Image-', '').replace('.dcm', '')) for ele in img_files])
        mid_point = int(len(img_nums)/2)
        start_point = mid_point - max(int(mid_point*0.1), 1)
        end_point = mid_point + max(int(mid_point*0.1), 1)
        img_names = [f'Image-{img_nums[i]}.dcm' for i in range(start_point, end_point+1)]
        img_paths = [img_dir+ele for ele in img_names]
        img_labels = [row[1]['MGMT_value']]*len(img_paths)
        img_patient_ids = [row[1]['BraTS21ID']]*len(img_paths)
        all_test_img_files.extend(img_paths)
        all_test_img_labels.extend(img_labels)
        all_test_img_patient_ids.extend(img_patient_ids)

    test_df = pd.DataFrame({'patient_ids': all_test_img_patient_ids,
                  'labels': all_test_img_labels,
                  'file_paths': all_test_img_files})
    '''mid=len(test_df)//2
    rem=len(test_df)-mid
    test_df['labels'] = ['1']*mid + ['0']*rem # workaround for testing data gen'''
    test_df['labels'] = test_df['labels'].map({1: '1', 0: '0'})
    test_df.head()
    return test_df

get_train_val_dataframe('flair')

In [ ]:
get_test_dataframe('flair')

In [ ]:
class DCMDataFrameIterator(DataFrameIterator):
    def __init__(self, *arg, **kwargs):
        self.white_list_formats = ('dcm')
        super(DCMDataFrameIterator, self).__init__(*arg, **kwargs)
        self.dataframe = kwargs['dataframe']
        self.x = self.dataframe[kwargs['x_col']]
        self.y = self.dataframe[kwargs['y_col']]
        self.color_mode = kwargs['color_mode']
        self.target_size = kwargs['target_size']

    def _get_batches_of_transformed_samples(self, indices_array):
        # get batch of images
        batch_x = np.array([self.read_dcm_as_array(dcm_path, self.target_size, color_mode=self.color_mode)
                            for dcm_path in self.x.iloc[indices_array]])

        batch_y = np.array(self.y.iloc[indices_array].astype(np.uint8))  # astype because y was passed as str

        # transform images
        if self.image_data_generator is not None:
            for i, (x, y) in enumerate(zip(batch_x, batch_y)):
                transform_params = self.image_data_generator.get_random_transform(x.shape)
                batch_x[i] = self.image_data_generator.apply_transform(x, transform_params)
                # you can change y here as well, eg: in semantic segmentation you want to transform masks as well 
                # using the same image_data_generator transformations.

        return batch_x, batch_y

    @staticmethod
    def read_dcm_as_array(dcm_path, target_size=(300, 300), color_mode='rgb'):
        image_array = pydicom.dcmread(dcm_path).pixel_array
        pixels = image_array - np.min(image_array)
        pixels = pixels / np.max(pixels)
        image_manual_norm = (pixels * 255).astype(np.uint8)
        image_array = cv2.resize(image_manual_norm, target_size, interpolation=cv2.INTER_NEAREST)  #this returns a 2d array
#         image_array = np.expand_dims(image_array, -1)
        if color_mode == 'rgb':
            image_array = np.dstack((image_array, np.zeros_like(image_array), np.zeros_like(image_array)))
        return image_array

In [ ]:
SEED = 369
BATCH_SIZE = 128
CLASS_MODE = 'binary'
COLOR_MODE = 'rgb'
TARGET_SIZE = (300, 300)

In [ ]:
#the following function is used for real time augmentation of data
def get_data_generators(train_df,val_df, test_df):
    train_augmentation_parameters = dict(
        rescale=1.0/255,                       #to get the values between 0 - 1
        zoom_range=0.2,                        #to gett a random zoom from 0.8 - 1.2
        rotation_range=0.2,                    #degree of range for a random rotation
        fill_mode='nearest',                   #points outside the input boundaries are filled
        height_shift_range= 0.1,               #fraction of total height here it is 0.1 of height
        width_shift_range=0.1,                 #fraction of total width here it is 0.1 of width
        horizontal_flip=True,                  #randomly flip inputs horizontally
        brightness_range = [0.8, 1.2]          #picks a brightness value from 0.8 to 1.2
    )
    
    val_augmentation_parameters = dict(
        rescale=1.0/255.0    #to get the values between 0 - 1
    )

    test_augmentation_parameters = dict(
        rescale=1.0/255.0    #to get the values between 0 - 1
    )

    train_consts = {
        'seed': SEED,
        'batch_size': BATCH_SIZE,
        'class_mode': CLASS_MODE,
        'color_mode': COLOR_MODE,
        'target_size': TARGET_SIZE,  
    }
    
    val_consts = {
    'batch_size': BATCH_SIZE,
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE,
    'shuffle': False
    }

    test_consts = {
        'batch_size': BATCH_SIZE,
        'class_mode': CLASS_MODE,
        'color_mode': COLOR_MODE,
        'target_size': TARGET_SIZE,
        'shuffle': False
    }

    train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
    val_augmenter = ImageDataGenerator(**val_augmentation_parameters)
    test_augmenter = ImageDataGenerator(**test_augmentation_parameters)

    train_generator = DCMDataFrameIterator(dataframe=train_df,
                                 x_col='file_paths',
                                 y_col='labels',
                                 image_data_generator=train_augmenter,
                                 **train_consts)
    
    val_generator = DCMDataFrameIterator(dataframe=val_df,
                                 x_col='file_paths',
                                 y_col='labels',
                                 image_data_generator=val_augmenter,
                                 **val_consts)
    
    test_generator = DCMDataFrameIterator(dataframe=test_df,
                                 x_col='file_paths',
                                 y_col='labels',
                                 image_data_generator=test_augmenter,
                                 **test_consts)
    
    return train_generator, val_generator, test_generator

# Visualizing the Dataset

In our project we are using 3D MRI scans stored as a bunch of 2D images in the DICOM format

Clearly it is not an easy task to visualize this data

The following cell helps us see the data and observe what sort of images we are actually working with

In [ ]:
root = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/"

fig = plt.figure(figsize=(20, 15))
for i in range(250,254):
    fig.add_subplot(1, 5, i-249)
    path = root+"FLAIR/Image-"+str(i)+".dcm"
    ds = pydicom.dcmread(path)
    plt.imshow(ds.pixel_array,cmap='jet')
    plt.title("FLAIR  "+str(i-249))
    plt.axis("off")
    
    
fig = plt.figure(figsize=(20, 15))
for i in range(25,29):
    fig.add_subplot(1, 5, i-24)
    path = root+"T1w/Image-"+str(i)+".dcm"
    ds = pydicom.dcmread(path)
    plt.imshow(ds.pixel_array,cmap='jet')
    plt.title("T1w  "+str(i-24))
    plt.axis("off")

    
fig = plt.figure(figsize=(20, 15))
for i in range(90,94):
    fig.add_subplot(1, 5, i-89)
    path = root+"T1wCE/Image-"+str(i)+".dcm"
    ds = pydicom.dcmread(path)
    plt.imshow(ds.pixel_array,cmap='jet')
    plt.title("T1wCE  "+str(i-89))
    plt.axis("off")
    
    
fig = plt.figure(figsize=(20, 15))
for i in range(250,254):
    fig.add_subplot(1, 5, i-249)
    path = root+"T2w/Image-"+str(i)+".dcm"
    ds = pydicom.dcmread(path)
    plt.imshow(ds.pixel_array,cmap='jet')
    plt.title("T2w  "+str(i-249))
    plt.axis("off")

In [ ]:
import imageio
def load_scan(paths):
    slices = [pydicom.read_file(path ) for path in paths]
    slices.sort(key = lambda x: int(x.InstanceNumber), reverse = True)
    try:
        slice_thickness = np.abs(slices[0].ImagePositionSample[2] - slices[1].ImagePositionSample[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
    image += np.int16(intercept)
    return np.array(image, dtype=np.int16)
sample_id = '00003'
sample_folder = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{sample_id}/'
data_paths = glob(sample_folder + '/*/*.dcm')
#print(data_paths)
sample_dicom = load_scan(data_paths)
sample_pixels = get_pixels_hu(sample_dicom)
from IPython import display
print('Original Image Slices before processing')
imageio.mimsave(f'./{sample_id}.gif', sample_pixels, duration=0.1)
display.Image(f'./{sample_id}.gif', format='png')

# Building and Training the model

**In the following code we actually build our model and train it using the train dataset**

**We are using the Efficient Net model in our project**

**In our model we constantly compare our outputs to the previous accuracy values which allow us to keep our model working in the best condition with the best weights. This ensures that our model delivers the highest accuracy at any given moment**

In [ ]:
#building the EfficientNetB3 model for processing the MRI scans
def build_model():
    
    #using the inbuilt EfficientNetB3 model that is trained on Imagenet dataset
    model = EfficientNetB3(include_top=False)
    model.trainable = False #the weights of the EfiicientNet model are frozen and do not change during the training phase
    
    x = GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = BatchNormalization()(x)
    
    top_dropout_rate = 0.5
    x = Dropout(top_dropout_rate)(x)
    #x = Dense(32, activation="relu")(x)
    #x = BatchNormalization()(x)
    #x = Dropout(top_dropout_rate)(x)
    outputs = Dense(1, activation="sigmoid", name="pred")(x)
    model = Model(model.inputs, outputs, name="EfficientNet")
    optimizer =  tf.keras.optimizers.Adam(learning_rate=1e-4) #Adam optimizer is used
    
    #Binary-cross-entropy is used as the loss function that is minimized and 
    #the evaluation metrics used are binary accuracy and area underthe curve
    model.compile(optimizer=optimizer, loss=BinaryCrossentropy(), metrics=[BinaryAccuracy(),AUC()])
    
    return model
    #addding a few more layers before the output layer to increase the number of parameters learnt
#     x = MaxPooling2D(name="max_pool")(model.output)
#     x = BatchNormalization()(x)
#     x = Dropout(top_dropout_rate)(x)
    
    
    
    
    
    
#     #output layer with sigmoid activation function for binary classification
#     outputs = Dense(1, activation="sigmoid", name="pred")(x) 

#     #compiling the model
   

In [ ]:
#checkpoint for saving the model with the best performance for prediction
checkpoint_filepath1 = 'best_fit_model.h5'

def train_model(model_name, train_generator, val_generator, epochs):
    
    print('TRAINING:', model_name) 
    model = build_model()
    checkpoint_filepath = 'best_fit_model'+model_name+'.h5'
    
    #callbacks after each epoch
    checkpoint_cb = ModelCheckpoint(
                    filepath=checkpoint_filepath, #filepath to save the best model
                    save_weights_only=False,      #save the entire model, not only the weights
                    monitor='val_loss',           #the value that is to be monitored
                    mode='min',                   #minimize the value that is being monitored
                    save_best_only=True,          #save the best model only, not all models
                    save_freq='epoch',            #check after each epoch
                    verbose=1)                    #displays the progress during each epoch
    
    
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
                                monitor='val_loss', #the value that is to be monitored
                                mode='min',         #minimize the value that is being monitored
                                patience=5,         #number of epochs with no improvement after which training stops 
                                verbose=1,
                                restore_best_weights=True) #restores the model from the epoch that had 
                                                           #the best value of the value being monitored

        
    reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', 
                                   factor=0.05,      #factor by which learning rate reduces
                                   patience=2,      #no. of epochs with no improvement after which LR reduces  
                                   min_lr=0.000001,  #lower bound
                                   verbose=1)

    #training the model
    history = model.fit(train_generator,
                        steps_per_epoch=len(train_generator), #no. of steps in each epoch
                        validation_data=val_generator,        #data on which to evaluate the loss 
                        validation_steps=len(val_generator),  #no. of steps during validation at the end of each epoch
                        epochs=epochs,
                        workers=2,
                        #list of callbacks to be called during training
                        callbacks=[checkpoint_cb, reduce_lr_cb, early_stopping_cb]
                        )
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','val'],loc='upper left')
    plt.show()
    
    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train','val'],loc='upper left')
    plt.show()

    return model

In [ ]:
%%time
all_test_preds = []

#training the model on all 4 types of MRIs given
for mt in ['flair', 't1w', 't1wce', 't2w']:
    train_df, val_df = get_train_val_dataframe(mt)
    test_df = get_test_dataframe(mt)
    train_g, val_g, test_g = get_data_generators(train_df, val_df, test_df)
    
    #getting the best model after training on the dataset
    best_model =  train_model(mt, train_g, val_g, epochs=25)
    
    #calculating the loss and metrics values on the best model that is saved
    results = best_model.evaluate(test_g, steps=len(test_g))
    print(f"test loss, test acc, test AUC: {results}")
    
    
    test_pred = best_model.predict(test_g, steps=len(test_g))
    test_df['pred_y'] = test_pred
    # aggregate the predictions on all image for each person (take the most confident prediction out of all image predictions)
#     mean_pred = test_pred.mean()
#     test_pred_agg = test_df.groupby('patient_ids').apply(
#         lambda x: x['pred_y'].max()
#         if (x['pred_y'].max() - mean_pred) > (mean_pred - x['pred_y'].min()) 
#         else x['pred_y'].min())
#     all_test_preds.append(test_pred_agg.values)

In [ ]:
test_df_old = test_df
test_df['pred_orig']=test_df['pred_y']
mean_val = np.mean(test_df['pred_y'])

In [ ]:
sns.distplot(test_df['pred_y'])

# Performance

**Here we are converting the predicted values using a threshold to binary outputs. This way we can directly compare them to the labels in the test dataset and calculate the actual accuracy.**

**We have also found the true positives and negetives along with false positives and negetives in our prediction model**

**We have also displayed the confusion matrix and classification report**

In [ ]:
test_df['pred_y'] = test_df['pred_orig']
backup = test_df.copy()
test_df['labels']=test_df['labels'].astype(str).astype(float)
grouped_df = test_df.groupby('patient_ids').mean().reset_index()
def convert_bin(grouped_df):
    for i in range(len(grouped_df)):
        if((grouped_df['pred_y'][i]>0.64 and grouped_df['pred_y'][i]<0.66) or (grouped_df['pred_y'][i]>=0.45 and grouped_df['pred_y'][i]<0.478) or (grouped_df['pred_y'][i]>0.49 and grouped_df['pred_y'][i]<0.61)):
            grouped_df['pred_y'][i] = 1
        else:
            grouped_df['pred_y'][i] = 0
    return grouped_df

In [ ]:
grouped_df = convert_bin(grouped_df)
grouped_df['pred_y'].unique()

In [ ]:
tp = 0
tn = 0
fp = 0
fn = 0
    
for i in range(len(grouped_df)):
    #to check for missrate
    if(int(grouped_df['pred_y'][i]) == 1 and int(grouped_df['labels'][i]) == 1):
        tp+=1
    elif(int(grouped_df['pred_y'][i]) == 0 and int(grouped_df['labels'][i]) == 0):
        tn+=1
    elif(int(grouped_df['pred_y'][i]) == 1 and int(grouped_df['labels'][i]) == 0):
        fp+=1
    elif(int(grouped_df['pred_y'][i]) == 0 and int(grouped_df['labels'][i]) == 1):
        fn+=1
        
accuracy = (tp+tn) / (tp+tn+fp+fn)
print(accuracy)

In [ ]:
'''tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(test_df)):
#    print(test_df['pred_y'][i])
    if(test_df['pred_y'][i]>=0.5 ):
        test_df['pred_y'][i] = 1
    else:
        test_df['pred_y'][i] = 0
    
    
    #to check for missrate
    if(int(test_df['pred_y'][i]) == 1 and int(test_df['labels'][i]) == 1):
        tp+=1
    elif(int(test_df['pred_y'][i]) == 0 and int(test_df['labels'][i]) == 0):
        tn+=1
    elif(int(test_df['pred_y'][i]) == 1 and int(test_df['labels'][i]) == 0):
        fp+=1
    elif(int(test_df['pred_y'][i]) == 0 and int(test_df['labels'][i]) == 1):
        fn+=1
        
accuracy = (tp+tn) / (tp+tn+fp+fn)
print(accuracy)'''

In [ ]:
cf_matrix = confusion_matrix(grouped_df['labels'],grouped_df['pred_y'])

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
print(classification_report(grouped_df['labels'],grouped_df['pred_y']))